In [31]:
import numpy as np
import struct
import pandas as pd
import matplotlib.pyplot as plt

def add_frame_nr(df,i,k,amount_frames):

    while i < amount_frames:
        if i == 0:
            iloc_1 = 0
        if i > 0:
            iloc_1 = k - 2907
        iloc_2 = k
        df.loc[iloc_1:iloc_2,'frames'] = int(i)
        k += 2907
        i += 1
    return df

def read_pc2(path):
    with open(path, 'rb') as f:
        head_fmt = '<12siiffi'
        data_fmt = '<fff'
        head_unpack = struct.Struct(head_fmt).unpack_from
        data_unpack = struct.Struct(data_fmt).unpack_from
        data_size = struct.calcsize(data_fmt)
        headerStr = f.read(struct.calcsize(head_fmt))
        head = head_unpack(headerStr)
        nverts, nframes = head[2], head[5]
        data = []
        for i in range(nverts*nframes):
            data_line = f.read(data_size)
            if len(data_line) != data_size:
                return None
            data.append(list(data_unpack(data_line)))
        data = np.array(data).reshape([nframes, nverts, 3])
    arr_reshaped = data.reshape(data.shape[0]*data.shape[1], data.shape[2])
    df = pd.DataFrame(arr_reshaped)
    df.columns = ['x_val2','y_val2','z_val2']
    df = add_frame_nr(df,0,2907,df.shape[0]/2907)

    return df


df2 = read_pc2(r'C:\files\KLA0057Animation_stabilised.pc2')



import plotly.graph_objects as go
import plotly.express as px
#df = pd.read_csv('C:\\files\\KLA0057Animation_stabilised.csv', sep = ';')

def select_frame (dataframe,framenr):
    return dataframe[dataframe['frames'] == framenr]


df2 = select_frame(df,0.0)

#import plotly.express as px
#fig = px.scatter_3d(newdf, x='x_val2', y='y_val2', z='z_val2')

#fig = go.Figure(data=[go.Mesh3d(x='x_val2', y='y_val2', z='z_val2', color='lightpink', opacity=0.50)])

fig = go.Figure(data=[go.Scatter3d(x=newdf['x_val2'], y=newdf['y_val2'], z=newdf['z_val2'],
                   opacity=0.8,mode='markers',
                  )])

fig.update_layout(
    scene = dict(
        xaxis = dict( range=[-100,100],),
                     yaxis = dict(range=[-500,500],),
                     zaxis = dict(range=[-100,100],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))


# tight layout
fig.show()

In [38]:
import os

directory = r'C:\files'
for filename in os.listdir(directory):
    if filename.endswith(".pc2"):
        name = filename[:-4]
        save_as_name = name = filename[:-4]+'.csv'
        filepath = 'C:\\files\\'+filename
        pc2 = read_pc2(filepath)    
        #pc2['test'] = np.ceil((pc2['Unnamed: 0'] /2907 ))
        pc2['row_num'] = np.arange(len(pc2))
        pc2['frames'] = pc2['row_num'] /2907
        pc2['frames'] = pc2['frames'].apply(np.floor)
        pc2.drop('row_num', axis= 1,inplace = True)
        pc2.to_csv(filepath + save_as_name,sep=";")
        print(filename)
        continue
    else:
        continue

print(len(pc2) / 2907)

KLA0057Animation_stabilised.pc2
72.0


In [4]:
pc2

,x_val2,y_val2,z_val2,frames
0,7.864367,-50.986713,-10.210601,0.0
1,8.256445,-51.688896,-13.339806,0.0
2,12.057677,-49.923397,-14.896049,0.0
3,12.312288,-48.962410,-10.753492,0.0
4,15.044912,-47.956661,-16.407492,0.0
...,...,...,...,...
209299,-4.962001,129.423752,-75.562759,71.0
209300,-5.005209,132.797928,-87.764526,71.0
209301,3.142819,-42.884651,-26.949648,71.0
209302,-1.012366,5.343492,-170.958725,71.0
